In [3]:
import src.data.breathe_data as br
import plotly.express as px
import src.modelling_fef2575.hfef2575 as hfef2575
import src.o2_fev1_analysis.smooth as smooth
import src.data.helpers as dh

In [4]:
# df = br.load_meas_from_excel("BR_O2_FEV1_FEF2575_PEF_inferred_AR_IA").drop(columns=["AR", "IA"])
# df = br.load_meas_from_excel("BR_O2_FEV1_FEF2575_PEF_inferred_AR_IA").drop(columns=["AR", "IA"])
# df = br.load_meas_from_excel("BR_O2_FEV1_FEF2575_PEF_Nan_processed")
# df = br.load_meas_from_excel("BR_O2_FEV1_FEF2575_PEF_inferred_AR_IA_processed").drop(
#     columns=["AR", "IA"]
# )
df = br.load_meas_from_excel("BR_O2_FEV1_FEF2575_PEF_inferred_AR_IA_HFEV1_HO2Sat").drop(
    columns=["AR", "IA", "HFEV1", "HO2Sat"]
)

In [5]:
df["PEF (L/s)"] = df.PEF / 60

In [6]:
def apply_new_smoothing(df, col, scale, shift):
    df[f"ec{col}"] = df[col]
    no_nan_mask = ~df[f"ec{col}"].isna()
    df.iloc[no_nan_mask] = smooth.identify_and_replace_outliers_up(
        df[no_nan_mask], f"ec{col}", scale=scale, shift=shift
    ).copy()
    df[f"ec{col}"][no_nan_mask] = smooth.smooth_vector(
        df[f"ec{col}"][no_nan_mask].to_numpy(), "max"
    )

    return df


df1 = (
    df.groupby(by="ID")
    .apply(lambda x: apply_new_smoothing(x, "FEV1", 3, 0.5))
    .reset_index(drop=True)
)
df1 = (
    df1.groupby(by="ID")
    .apply(lambda x: apply_new_smoothing(x, "FEF2575", 3, 0.5))
    .reset_index(drop=True)
)
df1 = (
    df1.groupby(by="ID")
    .apply(lambda x: apply_new_smoothing(x, "PEF (L/s)", 3, 1))
    .reset_index(drop=True)
)

ID 221 - Outlier up for ecFEV1, day 2022-08-09: 3.03 > 2.45 and > 2.44 (mean=1.94,std=0.17), update to 2.06
ID 244 - Outlier up for ecFEV1, day 2022-08-27: 4.38 > 3.87 and > 3.61 (mean=3.11,std=0.26), update to 3.06
ID 104 - Outlier up for ecFEF2575, day 2023-07-26: 1.73 > 1.72 and > 1.67 (mean=1.17,std=0.18), update to 1.12
ID 123 - Outlier up for ecFEF2575, day 2022-01-04: 5.66 > 5.65 and > 5.21 (mean=4.71,std=0.31), update to 4.94
ID 123 - Outlier up for ecFEF2575, day 2022-11-27: 5.45 > 5.38 and > 5.14 (mean=4.64,std=0.25), update to 4.55
ID 172 - Outlier up for ecFEF2575, day 2022-08-10: 4.23 > 4.05 and > 3.46 (mean=2.96,std=0.37), update to 2.92
ID 203 - Outlier up for ecFEF2575, day 2022-09-21: 2.25 > 1.98 and > 1.75 (mean=1.25,std=0.24), update to 1.20
ID 221 - Outlier up for ecFEF2575, day 2022-08-09: 3.46 > 2.96 and > 2.78 (mean=2.28,std=0.23), update to 2.44
ID 240 - Outlier up for ecFEF2575, day 2023-01-17: 4.64 > 4.43 and > 2.63 (mean=2.13,std=0.77), update to 1.87
ID 240 

In [7]:
print(df1.shape)
print(df.columns)
df1.head()

(15368, 22)
Index(['ID', 'Date Recorded', 'FEV1', 'O2 Saturation', 'FEF2575', 'PEF',
       'ecFEV1', 'Age', 'Sex', 'Height', 'Predicted FEV1',
       'Healthy O2 Saturation', 'ecFEV1 % Predicted', 'FEV1 % Predicted',
       'O2 Saturation % Healthy', 'AR mean', 'IA mean', 'HFEV1 mean',
       'HO2Sat mean', 'PEF (L/s)'],
      dtype='object')


,ID,Date Recorded,FEV1,O2 Saturation,FEF2575,PEF,ecFEV1,Age,Sex,Height,...,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,AR mean,IA mean,HFEV1 mean,HO2Sat mean,PEF (L/s),ecFEF2575,ecPEF (L/s)
0,101,2021-05-25,1.68,98,1.17,227,1.69,53,Male,173.0,...,46.813611,46.536607,100.874827,47.601368,0.654616,3.285826,97.643061,3.783333,1.17,3.933333
1,101,2021-05-26,1.65,98,1.06,236,1.69,53,Male,173.0,...,46.813611,45.705597,100.874827,47.601368,0.654616,3.285826,97.643061,3.933333,1.17,3.933333
2,101,2021-05-27,1.69,98,1.12,183,1.69,53,Male,173.0,...,46.813611,46.813611,100.874827,47.601368,0.654616,3.285826,97.643061,3.050000,1.12,3.933333
3,101,2021-05-28,1.67,98,1.08,175,1.69,53,Male,173.0,...,46.813611,46.259604,100.874827,47.601368,0.654616,3.285826,97.643061,2.916667,1.16,3.050000
4,101,2021-05-29,1.69,98,1.16,171,1.76,53,Male,173.0,...,48.752636,46.813611,100.874827,44.593062,0.667227,3.288958,97.621469,2.850000,1.16,3.116667


In [8]:
def calc_predicted_FEF2575_LMS_df(df):
    """
    Returns a Series with Predicted FEV1 from a DataFrame with Sex, Height, Age
    """
    df["Predicted FEF2575"] = df.apply(
        lambda x: hfef2575.calc_predicted_FEV1_LMS(
            hfef2575.load_LMS_spline_vals(x.Age, x.Sex),
            hfef2575.load_LMS_coeffs(x.Sex),
            x.Height,
            x.Age,
            x.Sex,
        )["M"],
        axis=1,
    )
    # df.apply(lambda x: sanity_checks.predicted_fev1(x["Predicted FEV1"], x.ID), axis=1)
    return df


def calc_FEVF2575_prct_predicted_df(df):
    """
    Returns input DataFrame with FEV1 % Predicted as a new column, after sanity check
    """
    df["ecFEF2575 % Predicted"] = df["ecFEF2575"] / df["Predicted FEF2575"] * 100
    df["FEF2575 % Predicted"] = df["FEF2575"] / df["Predicted FEF2575"] * 100
    # df.apply(
    #     lambda x: sanity_checks.fev1_prct_predicted(x["ecFEV1 % Predicted"], x.ID),
    #     axis=1,
    # )
    # df["FEV1 % Predicted"] = df["FEV1"] / df["Predicted FEV1"] * 100
    # df.apply(
    #     lambda x: sanity_checks.fev1_prct_predicted(x["FEV1 % Predicted"], x.ID), axis=1
    # )
    return df


# Add Healthy FEF2575
# df["Heatlhy FEF2575"] =
df1 = calc_predicted_FEF2575_LMS_df(df1)
df1 = calc_FEVF2575_prct_predicted_df(df1)

# Analysing PAP individuals to select CT scans candidates

In [ ]:
pap_ids = [
    "101",
    "102",
    "103",
    "104",
    "105",
    "106",
    "107",
    "108",
    "109",
    "110",
    "111",
    "112",
    "113",
    "114",
    "115",
    "116",
    "117",
    "118",
    "119",
    "120",
    "121",
    "122",
    "123",
    "124",
    "125",
    "126",
    "127",
    "128",
    "129",
    "130",
    "131",
    "132",
    "133",
    "134",
    "135",
    "136",
    "137",
    "138",
    "139",
    "140",
    "141",
    "142",
    "143",
    "144",
    "145",
    "146",
    "147",
    "148",
    "149",
    "150",
    "151",
    "152",
    "153",
    "154",
    "155",
    "156",
    "157",
    "158",
    "159",
    "160",
    "161",
    "162",
    "163",
    "164",
    "165",
    "166",
    "167",
    "168",
    "169",
    "170",
    "171",
    "172",
    "173",
    "174",
    "175",
    "176",
    "177",
    "178",
    "179",
    "180",
    "181",
    "182",
    "183",
    "184",
    "185",
    "186",
    "187",
    "188",
    "189",
    "190",
    "191",
    "192",
    "193",
    "194",
    "195",
    "196",
    "197",
    "198",
    "199",
    "200",
    "201",
    "202",
    "203",
    "204",
    "205",
    "206",
    "207",
    "208",
    "209",
    "210",
    "211",
    "212",
    "213",
    "214",
    "215",
    "216",
    "217",
    "218",
    "219",
    "220",
    "221",
    "222",
    "223",
    "224",
    "225",
    "226",
    "227",
    "228",
    "229",
    "230",
    "231",
    "232",
    "233",
    "234",
    "235",
    "236",
    "237",
    "238",
    "239",
    "240",
    "241",
    "242",
    "243",
    "244",
    "245",
    "246",
    "247",
    "248",
]

df2 = df1[df1.ID.isin(pap_ids)]

In [ ]:
import numpy as np
import pandas as pd


def get_avg_ar_ia(df):
    avg_ar = np.nanmean(df["AR mean"])
    avg_ia = np.nanmean(df["IA mean"])
    return avg_ar, avg_ia


dftmp = df2.groupby(by="ID").apply(get_avg_ar_ia)

res = (
    dftmp.apply(pd.Series)
    .reset_index()
    .rename(columns={0: "avg ar", 1: "avg ia"})
    # .drop(columns="level_1")
)

# plot avg ar vs avg ia
fig = px.scatter(res, x="avg ar", y="avg ia", color="ID")
fig.show()

In [ ]:
fig = px.scatter(df2, x="AR mean", y="IA mean", color="ID")
fig.update_layout(width=2000)
fig.show()

## PEF, FEF2575 data quality

In [9]:
df1.columns

Index(['ID', 'Date Recorded', 'FEV1', 'O2 Saturation', 'FEF2575', 'PEF',
       'ecFEV1', 'Age', 'Sex', 'Height', 'Predicted FEV1',
       'Healthy O2 Saturation', 'ecFEV1 % Predicted', 'FEV1 % Predicted',
       'O2 Saturation % Healthy', 'AR mean', 'IA mean', 'HFEV1 mean',
       'HO2Sat mean', 'PEF (L/s)', 'ecFEF2575', 'ecPEF (L/s)',
       'Predicted FEF2575', 'ecFEF2575 % Predicted', 'FEF2575 % Predicted'],
      dtype='object')

In [57]:
# Using plotly create histogram of fef2575%pef
def pef_for_df(df):
    fig = px.histogram(df, x="PEF", nbins=500, color="ID")
    # fig = px.histogram(df1, x="FEF2575 % Predicted", nbins=500)
    fig.update_layout(width=1500, height=700)
    fig.show()
    return


ids = df1[df1.PEF == 300].ID.unique()
print(ids)
pef_for_df(df1)
# pef_for_df(df1[df1.ID.isin(ids)])
# df1[df1.ID.isin(ids)].groupby("ID").apply(pef_for_df)

['104' '106' '107' '112' '116' '122' '158' '162' '176' '202' '220' '221'
 '224' '229' '238' '265' '272' '310' '334']


In [ ]:
df[df.PEF < 80]

In [ ]:
df["FEF2575%PEF"] = df.FEF2575 / df["PEF (L/s)"]

# Using plotly create histogram of fef2575%pef
fig = px.histogram(df, x="FEF2575%PEF")
fig.show()

In [ ]:
# Why are there measurements > 100? even > 70?

# If the test is well performed, FEF2575 should never be > than PEF
# Or the PEF is super low
print(len(df[df["FEF2575%PEF"] > 70]) / len(df) * 100, "% are > 70%")

df[df["FEF2575%PEF"] > 100]

In [ ]:
# PEF should be greater than FEV1
idx = df[df["PEF (L/s)"] < df["FEV1"]].index
df[df["PEF (L/s)"] < df["FEV1"]]

In [ ]:
# df1 = df1.drop(idx, axis=0, inplace=False)

In [ ]:
# FEF2575 should be lower than FEV1, otherwise:
# Test not well performed: PEF reached late -> flow increase over a long time (0.5s-0.1s) instead of 0.1s, little dynamic compression of the airways
df[df.FEV1 < df.FEF2575].sort_values(by="FEV1")

# Highlight HFEV1 postior shift

In [ ]:
df1["ecFEV1%HFEV1"] = df1["ecFEV1"] / df1["HFEV1 mean"] * 100

In [ ]:
# fig = px.scatter(df1, x="AR mean", y="ecFEV1%HFEV1", color="ID")
fig = px.scatter(df1, x="AR mean", y="ecFEV1 % Predicted", color="ID")
fig.update_traces(marker=dict(size=2))
fig.update_layout(width=1000, height=700)
fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
# Add ecFEV1%HFEV1 AR mean trace
fig.add_trace(
    go.Scatter(
        x=df1["AR mean"],
        y=df1["ecFEV1%HFEV1"],
        mode="markers",
        marker=dict(size=2),
        name="ecFEV1%HFEV1",
    )
)
# Add ecFEV1 % Predicted AR mean trace
fig.add_trace(
    go.Scatter(
        x=df1["AR mean"],
        y=df1["ecFEV1 % Predicted"],
        mode="markers",
        marker=dict(size=2),
        name="ecFEV1 % Predicted",
    )
)
fig.update_layout(width=1000, height=700)
# x axis title
fig.update_xaxes(title_text="Airway resistance (%)")
fig.update_yaxes(title_text="FEV1% (prior, posterior)")
fig.show()

# Modelling FEF2575

In [1]:
# fig = px.scatter(df1, x="FEV1", y="FEF2575", color="ID")
fig = px.scatter(df1, x="ecFEV1", y="ecFEF2575", color="ID")
fig.update_traces(marker=dict(size=3))
fig.update_layout(width=800, height=800)
fig.show()

NameError: name 'px' is not defined

In [22]:
def plot_for_df(df):
    df = df.reset_index()
    # fig = px.scatter(df, x="ecPEF (L/s)", y="ecFEF2575", color="ID")
    # fig = px.scatter(df1, x="PEF (L/s)", y="FEF2575", color="ID")
    fig.update_traces(marker=dict(size=3))
    # Update the layout to show othonormal axises
    fig.update_xaxes(range=[0, 17])
    fig.update_yaxes(range=[0, 7.5])
    fig.update_layout(width=1200, height=600)
    # fig.write_image(f"{dh.get_path_to_main()}PlotsBreathe/AR_modelling/{df.ID[0]}.pdf")
    fig.show()
    return


# df1.groupby(by="ID").apply(plot_for_df)
plot_for_df(df1)

In [19]:
df1["FEF2575%FEV1"] = df1["FEF2575"] / df1["FEV1"] * 100
df1["ecFEF2575%ecFEV1"] = df1["ecFEF2575"] / df1["ecFEV1"] * 100
df1["FEF2575%PEF"] = df1["FEF2575"] / df1["PEF (L/s)"] * 100
df1["ecFEF2575%ecPEF"] = df1["ecFEF2575"] / df1["ecPEF (L/s)"] * 100

In [ ]:
fig = px.scatter(df1, x="AR mean", y="FEF2575%FEV1", color="ID")
fig.update_traces(marker=dict(size=2))
fig.update_layout(width=1200, height=800)
# Update y axis ranve
fig.update_yaxes(range=[0, 250])
fig.show()

In [ ]:
fig = px.scatter(df1, x="AR mean", y="FEF2575%PEF", color="ID")
fig.update_traces(marker=dict(size=2))
fig.update_layout(width=1200, height=800)
fig.show()

In [ ]:
print(max(df1["IA mean"]))

In [2]:
threshold = 1.7
# fig = px.scatter(df1, x="AR mean", y="FEF2575%PEF", color="IA mean")
fig = px.scatter(df1, x="AR mean", y="ecFEF2575%ecPEF", color="IA mean")
# fig = px.scatter(df1, x="AR mean", y="ecFEF2575%ecFEV1", color="IA mean")
# fig = px.scatter(df1, x="AR mean", y="FEF2575%FEV1", color="IA mean")
# Update the scale of the figure's color heatmap
t1 = 1.6
t2 = 2
t3 = 8
colorscale = [
    [0, "#FFFFA1"],
    [round(t1 / max(df1["IA mean"]), 2) * 0.99, "#FFFFA1"],
    [round(t1 / max(df1["IA mean"]), 2), "#FFFFA1"],
    [round(t2 / max(df1["IA mean"]), 2) * 0.99, "#FFFFA1"],
    [round(t2 / max(df1["IA mean"]), 2), "orange"],
    [round(t3 / max(df1["IA mean"]), 2) * 0.99, "orange"],
    [round(t3 / max(df1["IA mean"]), 2), "black"],
    [1, "black"],
]
fig.update_coloraxes(colorscale=colorscale)
# fig.update_yaxes(range=[0,150])
fig.update_traces(marker=dict(size=3))
fig.update_layout(width=1200, height=800)
fig.show()

NameError: name 'px' is not defined

In [ ]:
fig = px.scatter(df1, x="AR mean", y="ecFEF2575 % Predicted", color="ID")
fig.update_yaxes(range=[0, 250])
fig.update_traces(marker=dict(size=2))
fig.update_layout(width=1200, height=800)
fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
# Add ecFEV1%HFEV1 AR mean trace
fig.add_trace(
    go.Scatter(
        x=df1["AR mean"],
        y=df1["ecFEF2575 % Predicted"],
        mode="markers",
        marker=dict(size=2),
        name="ecFEF2575 % Predicted",
    )
)
# Add ecFEV1 % Predicted AR mean trace
fig.add_trace(
    go.Scatter(
        x=df1["AR mean"],
        y=df1["ecFEV1 % Predicted"],
        mode="markers",
        marker=dict(size=2),
        name="ecFEV1 % Predicted",
    )
)
fig.update_layout(width=1000, height=700)
# x axis title
fig.update_xaxes(title_text="Airway resistance (%)")
fig.update_yaxes(title_text="Lung function")
fig.show()

In [ ]:
fig = px.scatter(df1, x="IA mean", y="ecFEF2575 % Predicted", color="ID")
fig.update_traces(marker=dict(size=2))
fig.update_layout(width=1200, height=800)
fig.show()

In [ ]:
fig = px.scatter(df1, x="IA mean", y="FEF2575%PEF", color="ID")
fig.update_traces(marker=dict(size=2))
fig.update_layout(width=1200, height=800)
fig.show()

In [ ]:
fig = px.scatter(df1, x="IA mean", y="FEV1", color="ID")
fig.update_traces(marker=dict(size=2))
fig.update_layout(width=1200, height=800)
fig.show()